# VAMPIRES IMR to FLC Mueller Matrix

This notebook computes the Mueller matrix for a complex optical train using the full capabilities of the pyMuellerMat package. The optical train is defined as a system dictionary, with each component's type and properties (including rotation angles and AOIs) specified directly. The system is built and evaluated using pyMuellerMat's `generate_system_mueller_matrix` and `update_system_mm` functions.

You can specify the rotation angle of each QWP and the angle of incidence for each dichroic and periscope mirror in the system dictionary below. The notebook will compute the total Mueller matrix and extract the net retardance, diattenuation, and rotation angle.

In [1]:
import sys
sys.path.append('/home/rebeccaz/Github/vampires_calibration')

import numpy as np
from pyMuellerMat.common_mm_functions import *
from pyMuellerMat import common_mms as cmm
from pyMuellerMat import MuellerMat
from instruments import generate_system_mueller_matrix, update_system_mm


In [2]:
# === USER SETTINGS: Define Optical Train as System Dictionary ===
wavelength_nm = 675  # Wavelength in nm

# Example indices of refraction (adjust as needed)
n_dichroic = 2.0 + 0.0j  # Dielectric dichroic (guess)
n_mirror = 1.73 + 1j * 7.94  # Aluminum at 675 nm

# User-specified angles (degrees)
aoi_dichroic_ao3k = 45
qwp1_angle = 128
qwp2_angle = 168
aoi_dichroic1 = 45
aoi_periscope1 = 45 # from refractiveindex.info for aluminum at 675 nm, 45 degree AOI
aoi_periscope2 = 45 # from refractiveindex.info for aluminum at 675 nm, 45 degree AOI
aoi_dichroic2 = 30

# Mirror Fresnel coefficients (example, from aluminum_mirrors_mueller_matrices)
R_s_periscope = RS = 0.93731 # from refractiveindex.info for aluminum at 675 nm, 45 degree AOI
R_p_periscope = 0.87855 # from refractiveindex.info for aluminum at 675 nm, 45 degree AOI
delta_mirror = np.radians(22.185 - (-168.907))  # from φp − φs

# === Dichroic Approximate Fresnel Coefficients (Visible/NIR, AOI-dependent) ===
# AO3K_Dichroic and Dichroic1: AOI = 45 deg (typical design AOI)
# Dichroic2: AOI = 30 deg (lower AOI, typically higher reflectance and lower phase difference)
# See:
# - Semrock technical note: https://www.semrock.com/Data/Sites/1/semrockpdfs/tn_dichroic_angle_dependence.pdf
# - Chroma dichroic specs: https://www.chroma.com/products/optical-filters/dichroic-beamsplitters
# - Newport: https://www.newport.com/g/dichroic-mirrors

# AOI = 45 deg (AO3K_Dichroic, Dichroic1)
R_s_dichroic_45 = 0.98
R_p_dichroic_45 = 0.97
delta_dichroic_45 = np.radians(5.0)  # phase difference (approximate)

# AOI = 30 deg (Dichroic2)
R_s_dichroic_30 = 0.99  # higher reflectance at lower AOI
R_p_dichroic_30 = 0.985
delta_dichroic_30 = np.radians(2.0)  # phase difference even smaller at lower AOI

# Define the system as a dictionary (downstream to upstream)
system_dict = {
    "components": {
        "Dichroic2": {
            "type": "diattenuator_retarder_function2",
            "properties": {"r1": R_s_dichroic_30, "r2": R_p_dichroic_30, "delta": delta_dichroic_30}
        },
        "PeriscopeBlock": {
            "type": "arbitrary_matrix_function",
            "properties": {"mm": np.array([
                [0.46482, -0.46482, 0.00000, 0.00000],
                [0.46482, -0.46482, 0.00000, 0.00000],
                [0.00000, 0.00000, 0.00000, 0.00000],
                [0.00000, 0.00000, 0.00000, 0.00000],
            ])}
        },
        "QWP2": {
            "type": "general_retarder_function",
            "properties": {"phi": np.pi/2, "theta": qwp2_angle}
        },
        "QWP1": {
            "type": "general_retarder_function",
            "properties": {"phi": np.pi/2, "theta": qwp1_angle}
        },
        "Dichroic1": {
            "type": "diattenuator_retarder_function2",
            "properties": {"r1": R_s_dichroic_45, "r2": R_p_dichroic_45, "delta": delta_dichroic_45}
        },
        "AO3K_Dichroic": {
            "type": "diattenuator_retarder_function2",
            "properties": {"r1": R_s_dichroic_45, "r2": R_p_dichroic_45, "delta": delta_dichroic_45}
        },
    }
}

In [3]:
# === Build System Mueller Matrix using pyMuellerMat ===
system_mm = generate_system_mueller_matrix(system_dict)
M_total = system_mm.evaluate()

print("=== PeriscopeBlock Properties ===")
for key, value in system_dict["components"]["PeriscopeBlock"]["properties"].items():
    print(f"{key}: {value}")

# Display the resulting Mueller matrix
print("=== Optical Train Mueller Matrix (pyMuellerMat System) ===")
print(M_total)


=== PeriscopeBlock Properties ===
mm: [[ 0.46482 -0.46482  0.       0.     ]
 [ 0.46482 -0.46482  0.       0.     ]
 [ 0.       0.       0.       0.     ]
 [ 0.       0.       0.       0.     ]]
=== Optical Train Mueller Matrix (pyMuellerMat System) ===
[[ 0.4394052   0.19392196 -0.04441941 -0.39178836]
 [ 0.4394052   0.19392196 -0.04441941 -0.39178836]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]]


In [4]:
# === Evaluate and Display System Mueller Matrix ===
print("=== Optical Train Mueller Matrix (pyMuellerMat System) ===")
print(M_total)

=== Optical Train Mueller Matrix (pyMuellerMat System) ===
[[ 0.4394052   0.19392196 -0.04441941 -0.39178836]
 [ 0.4394052   0.19392196 -0.04441941 -0.39178836]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]]


In [5]:
# === Extract and Display Retardance, Diattenuation, and Rotation Angle (System) ===
def compute_diattenuation(M):
    return np.abs(M[0, 1]) / M[0, 0]

def compute_retardance(M):
    m22 = M[2:4, 2:4]
    trace = np.trace(m22)
    return np.real(np.arccos(trace / 2))

def compute_rotation_angle(M):
    return 0.5 * np.degrees(np.arctan2(M[1, 0], M[0, 0]))

ret = compute_retardance(M_total)
dia = compute_diattenuation(M_total)
rot = compute_rotation_angle(M_total)

print("\n=== Optical Train Properties (pyMuellerMat System) ===")
print(f"Retardance:     {ret / (2 * np.pi):.4f} waves")
print(f"Diattenuation:  {dia:.6f}")
print(f"Rotation Angle: {rot:.2f} degrees")


=== Optical Train Properties (pyMuellerMat System) ===
Retardance:     0.2500 waves
Diattenuation:  0.441328
Rotation Angle: 22.50 degrees
